In [1]:
import pandas as pd
import re

### Load the datasets

In [2]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", compression = 'gzip', 
                    sep = ';', encoding = 'latin-1', low_memory=False)
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
ratings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz", compression = 'gzip',
                      sep = ';', encoding = 'latin-1', low_memory=False)
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz", compression = 'gzip', 
                    sep = ';', encoding = 'latin-1', low_memory=False)
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
goodbooks = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz", compression = 'gzip', 
                        encoding = 'latin-1', low_memory=False)
goodbooks.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [6]:
goodbooks_ratings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz", 
                                compression = 'gzip', encoding = 'latin-1', low_memory=False)
goodbooks_ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


### 1) Normalize the location field of Users dataset, splitting into city, region, country

In [19]:
users[['City', 'Region', 'Country']] = users['Location'].str.split(',', expand = True, n=2)
users.drop('Location', axis = 1, inplace = True)
users.head()

,User-ID,Age,City,Region,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom


### 2) For each book in the Books dataset, compute its average rating.

In [24]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [25]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [26]:
books_w_ratings = pd.merge(books, ratings, on='ISBN', how='left')
books_w_ratings.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0


In [27]:
books_avg = books_w_ratings.groupby(['ISBN', 'Book-Title'], as_index=False)['Book-Rating'].mean()
books_avg.head(15)

,ISBN,Book-Title,Book-Rating
0,0000913154,The Way Things Work: An Illustrated Encycloped...,8.0
1,0001010565,Mog's Christmas,0.0
2,0001046438,Liar,9.0
3,0001046713,Twopence to Cross the Mersey,0.0
4,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",6.0
5,0001046934,The Prime of Miss Jean Brodie,0.0
6,0001047213,The Fighting Man,9.0
7,0001047647,First Among Equals,0.0
8,0001047663,Matter Of Honour,0.0
9,0001047868,Kidnapped,0.0


### 3) For each book in the GoodBooks dataset, compute its average rating.

In [30]:
goodbooks.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [31]:
goodbooks[['book_id', 'original_title', 'average_rating']]

,book_id,original_title,average_rating
0,1,The Hunger Games,4.34
1,2,Harry Potter and the Philosopher's Stone,4.44
2,3,Twilight,3.57
3,4,To Kill a Mockingbird,4.25
4,5,The Great Gatsby,3.89
...,...,...,...
94,95,The Picture of Dorian Gray,4.06
95,96,Fifty Shades Freed,3.88
96,97,Dracula,3.98
97,98,Flickan som lekte med elden,4.22


### 4) Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset merged books. The books that have not been merged together will not appear in merged books.

In [36]:
merged = books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).size()
merged.head()

,Book-Title,Book-Author,Publisher,size
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,1
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,1
2,Apple Magic (The Collector's series),Martina Boudreau,Amer Cooking Guild,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,Zonderkidz,1
4,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,"Teleonet, Incorporated",1


In [37]:
merged_books = merged[merged['size']>1]
merged_books.head()

,Book-Title,Book-Author,Publisher,size
41,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
59,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
255,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
465,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
713,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


In [38]:
len(merged_books)

4725

### 5) For each book in merged books compute its average rating.

In [39]:
merged_books_w_avg = pd.merge(merged_books, books_avg, on="Book-Title")
merged_books_w_avg.head()

,Book-Title,Book-Author,Publisher,size,ISBN,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920317,6.00
1,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920465,0.00
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,0.00
3,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792277295,3.40
4,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196256,7.25


In [40]:
merged_books_w_avg.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False)['Book-Rating'].mean()

,Book-Title,Book-Author,Publisher,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,3.000000
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,1.700000
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,6.958333
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3.111111
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.000000
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,1.859048
4721,Zia,Scott O'Dell,Yearling Books,1.859048
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,2.000000
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000


### 6) For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.

In [41]:
merged_books_w_ratings = pd.merge(merged_books, books_w_ratings, on = ['Book-Title', 'Book-Author', 'Publisher'])
merged_books_w_ratings.head()

,Book-Title,Book-Author,Publisher,size,ISBN,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920465,1994,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,http://images.amazon.com/images/P/1565920465.0...,271245.0,0.0
1,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920317,1993,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,http://images.amazon.com/images/P/1565920317.0...,87141.0,6.0
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,2000,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,79186.0,0.0
3,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,2000,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,114216.0,0.0
4,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,2000,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,http://images.amazon.com/images/P/0792276833.0...,201058.0,0.0


In [42]:
merged_books_w_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False)['Book-Rating'].max()

,Book-Title,Book-Author,Publisher,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.0
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,9.0
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,10.0
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,10.0
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.0
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,10.0
4721,Zia,Scott O'Dell,Yearling Books,8.0
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,8.0
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.0


In [43]:
merged_books_w_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False)['Book-Rating'].min()

,Book-Title,Book-Author,Publisher,Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,0.0
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,0.0
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,4.0
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,0.0
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.0
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,0.0
4721,Zia,Scott O'Dell,Yearling Books,0.0
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,0.0
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.0


### 7) For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.

In [44]:
goodbooks.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [107]:
gb_authors = {}

def gb_diz(authors, count):
    for i in range(len(authors)):
        occ = authors[i].split(',')
        for elem in occ:
            if elem in gb_authors:
                gb_authors[elem] += count[i]/len(occ)
            else:
                gb_authors[elem] = count[i]/len(occ)

In [108]:
authors = list(goodbooks['authors'])
work_text_reviews_count = list(goodbooks['work_text_reviews_count'])

In [109]:
gb_diz(authors, work_text_reviews_count)

In [110]:
gb_authors

{'Suzanne Collins': 340066.0,
 'J.K. Rowling': 136668.0,
 ' Mary GrandPrÃ©': 136668.0,
 'Stephenie Meyer': 258573.0,
 'Harper Lee': 72586.0,
 'F. Scott Fitzgerald': 51992.0,
 'John Green': 230584.0,
 'J.R.R. Tolkien': 52986.0,
 'J.D. Salinger': 44920.0,
 'Dan Brown': 66672.0,
 'Jane Austen': 52994.0,
 'Khaled Hosseini': 103375.0,
 'Veronica Roth': 156896.0,
 'George Orwell': 50644.666666666664,
 ' Erich Fromm': 15172.666666666666,
 ' CelÃ¢l Ã\x9cster': 15172.666666666666,
 'Anne Frank': 6941.666666666667,
 ' Eleanor Roosevelt': 6941.666666666667,
 ' B.M. Mooyaart-Doubleday': 6941.666666666667,
 'Stieg Larsson': 46983.0,
 ' Reg Keeland': 46983.0,
 ' Rufus Beck': 12033.0,
 'Alice Sebold': 36642.0,
 'William Golding': 26886.0,
 'William Shakespeare': 7389.0,
 ' Robert           Jackson': 7389.0,
 'Gillian Flynn': 121614.0,
 'Kathryn Stockett': 78204.0,
 'John Steinbeck': 24642.0,
 'Arthur Golden': 25605.0,
 'E.L. James': 128776.0,
 'Paulo Coelho': 27890.5,
 ' Alan R. Clarke': 27890.5,
 'L

In [119]:
authors_reviews = pd.DataFrame.from_dict(gb_authors, orient = 'index', columns = ['shared number of reviews with a text'])
authors_reviews

,shared number of reviews with a text
Suzanne Collins,340066.000000
J.K. Rowling,136668.000000
Mary GrandPrÃ©,136668.000000
Stephenie Meyer,258573.000000
Harper Lee,72586.000000
...,...
Oscar Wilde,9823.500000
Jeffrey Eugenides,9823.500000
Bram Stoker,5754.333333
Nina Auerbach,5754.333333


### 8) For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

### 9) Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [46]:
both = pd.merge(goodbooks[['book_id', 'isbn', 'authors', 'title', 'average_rating']], books_avg[['ISBN', 'Book-Rating']], 
                left_on = 'isbn', right_on = 'ISBN')
both

,book_id,isbn,authors,title,average_rating,ISBN,Book-Rating
0,11,1594480001,Khaled Hosseini,The Kite Runner,4.26,1594480001,2.400000
1,18,043965548X,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53,043965548X,3.533333
2,60,1400032717,Mark Haddon,The Curious Incident of the Dog in the Night-Time,3.85,1400032717,4.813187
3,75,014028009X,Helen Fielding,"Bridget Jones's Diary (Bridget Jones, #1)",3.75,014028009X,3.751852
4,90,014038572X,S.E. Hinton,The Outsiders,4.06,014038572X,4.460674


In [47]:
both['avg_difference'] = both['average_rating']-both['Book-Rating']
both

,book_id,isbn,authors,title,average_rating,ISBN,Book-Rating,avg_difference
0,11,1594480001,Khaled Hosseini,The Kite Runner,4.26,1594480001,2.400000,1.860000
1,18,043965548X,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53,043965548X,3.533333,0.996667
2,60,1400032717,Mark Haddon,The Curious Incident of the Dog in the Night-Time,3.85,1400032717,4.813187,-0.963187
3,75,014028009X,Helen Fielding,"Bridget Jones's Diary (Bridget Jones, #1)",3.75,014028009X,3.751852,-0.001852
4,90,014038572X,S.E. Hinton,The Outsiders,4.06,014038572X,4.460674,-0.400674


### 10) Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [48]:
users.head()

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom


In [49]:
users_nan = users[users['Age'].isnull()]
users_nan

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom
6,7,"washington, dc, usa",NaN,washington,dc,usa
7,8,"timmins, ontario, canada",NaN,timmins,ontario,canada
...,...,...,...,...,...,...
278849,278850,"sergnano, lombardia, italy",NaN,sergnano,lombardia,italy
278853,278854,"portland, oregon, usa",NaN,portland,oregon,usa
278855,278856,"brampton, ontario, canada",NaN,brampton,ontario,canada
278856,278857,"knoxville, tennessee, usa",NaN,knoxville,tennessee,usa


In [50]:
users_temp = users[users['Age'].notnull()]
users_temp

,User-ID,Location,Age,City,Region,Country
1,2,"stockton, california, usa",18.0,stockton,california,usa
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
5,6,"santa monica, california, usa",61.0,santa monica,california,usa
9,10,"albacete, wisconsin, spain",26.0,albacete,wisconsin,spain
10,11,"melbourne, victoria, australia",14.0,melbourne,victoria,australia
...,...,...,...,...,...,...
278848,278849,"georgetown, ontario, canada",23.0,georgetown,ontario,canada
278850,278851,"dallas, texas, usa",33.0,dallas,texas,usa
278851,278852,"brisbane, queensland, australia",32.0,brisbane,queensland,australia
278852,278853,"stranraer, n/a, united kingdom",17.0,stranraer,n/a,united kingdom


In [51]:
users_temp['Age_group'] = pd.cut(users_temp['Age'], bins = [0, 15, 25, 35, 45, 55, 65, 75, 85, 95, 1000], right = False,
                                labels = ['<15', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84', '85-94', '>95'])
users_temp

<ipython-input-51-64c3c3f37d7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_temp['Age_group'] = pd.cut(users_temp['Age'], bins = [0, 15, 25, 35, 45, 55, 65, 75, 85, 95, 1000], right = False,


,User-ID,Location,Age,City,Region,Country,Age_group
1,2,"stockton, california, usa",18.0,stockton,california,usa,15-24
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal,15-24
5,6,"santa monica, california, usa",61.0,santa monica,california,usa,55-64
9,10,"albacete, wisconsin, spain",26.0,albacete,wisconsin,spain,25-34
10,11,"melbourne, victoria, australia",14.0,melbourne,victoria,australia,<15
...,...,...,...,...,...,...,...
278848,278849,"georgetown, ontario, canada",23.0,georgetown,ontario,canada,15-24
278850,278851,"dallas, texas, usa",33.0,dallas,texas,usa,25-34
278851,278852,"brisbane, queensland, australia",32.0,brisbane,queensland,australia,25-34
278852,278853,"stranraer, n/a, united kingdom",17.0,stranraer,n/a,united kingdom,15-24


In [52]:
grouped = users_temp.groupby(users_temp.Age_group)

x = list(grouped.groups)
df = {}

for elem in x:
    df[elem] = grouped.get_group(elem)
    
df['>95']

,User-ID,Location,Age,City,Region,Country,Age_group
1288,1289,"san jose, california, usa",103.0,san jose,california,usa,>95
1322,1323,"milano, lombardia, italy",104.0,milano,lombardia,italy,>95
1578,1579,"akure, ondo/nigeria, nigeria",231.0,akure,ondo/nigeria,nigeria,>95
3084,3085,"zÃ¼rich, switzerland, switzerland",104.0,zÃ¼rich,switzerland,switzerland,>95
3210,3211,"le mesnil saint denis, yvelines, france",119.0,NaN,NaN,NaN,>95
...,...,...,...,...,...,...,...
276352,276353,"hillsdale, new york, usa",104.0,hillsdale,new york,usa,>95
277107,277108,"quinto, ticino, switzerland",104.0,quinto,ticino,switzerland,>95
277503,277504,"san diego, california, usa",103.0,san diego,california,usa,>95
277558,277559,"lake george, new york, usa",98.0,lake george,new york,usa,>95


### 11) Find the books that appear only in the goodbooks datasets.

In [53]:
only_goodbooks = pd.merge(goodbooks, books['ISBN'], left_on = 'isbn', right_on = 'ISBN', how = 'left')
only_goodbooks[only_goodbooks['ISBN'].isnull()]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,ISBN
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,NaN
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,NaN
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,NaN
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"Oscar Wilde, Jeffrey Eugenides",1891.0,The Picture of Dorian Gray,...,674551,19647,9825,30735,129420,246892,257679,https://images.gr-assets.com/books/1424596966m...,https://images.gr-assets.com/books/1424596966s...,NaN
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,E.L. James,2012.0,Fifty Shades Freed,...,610116,25287,39079,55691,109963,140304,265079,https://images.gr-assets.com/books/1336418837m...,https://images.gr-assets.com/books/1336418837s...,NaN
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"Bram Stoker, Nina Auerbach, David J. Skal",1897.0,Dracula,...,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,NaN
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"Stieg Larsson, Reg Keeland",2006.0,Flickan som lekte med elden,...,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,NaN


### 12) Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [54]:
pairs = books.groupby(['Book-Title', 'Book-Author'], as_index = False).size()
pairs.loc[pairs['size'].idxmax()]

Book-Title          Little Women
Book-Author    Louisa May Alcott
size                          21
Name: 113850, dtype: object

### 13) Find the author with the highest average ratio according to the goodbooks datasets.